<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/APN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Data/Tweets_clean_data.csv')
df = df.sample(frac=1)

In [5]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])

In [6]:
print(data_tweet[:5])
print(data_cat[:5])

[['lol', 'no', 'mention', 'of', 'pak', 'pm', 'or', 'even', 'his', 'speech', 'on', 'any', 'international', 'news', 'channel', 'and', 'pakis', 'are', 'rejoicing', 'as', 'if', 'the', 'world', 'stands', 'with', 'them'], ['hope', 'they', 'refuse', 'frown', 'sad', 'andry', 'or', 'pouting', 'x', 'depressing'], ['thanks', 'for', 'ripping', 'me', 'off', 'again', 'luthansa', '€400', 'not', 'enough', 'for', 'a', 'one', 'way', 'flight', 'to', 'man', 'from', 'frk', 'then', '€30', 'for', 'a', 'bag', 'then', 'free', 'at', 'gate'], ['you', 'sound', 'like', 'my', 'husband', 'he', 'makes', 'hotter', 'and', 'hotter', 'sauce', 'the', 'fiery', 'torment', 'brings', 'him', 'mouth', 'joy', 'fire'], ['giacchino', 'will', 'this', 'be', 'aired', 'on', 'radio', 'or', 'filmed', 'lots', 'of', 'fans', 'over', 'the', 'pond', 'too', 'wales', 'lost']]
[[0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]]


In [7]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [8]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(475595, 595345)

In [9]:

positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])

print(positive)
print(negative)

['abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'agreeabl

In [10]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [11]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
mul_factor = 1000

In [12]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i)*mul_factor if (i in vocab and nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ' and i in positive) else w2v.wv.get_vector(i)*mul_factor*(-1) if (i in vocab and nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ' and i in negative) else w2v.wv.get_vector(i)*mul_factor if (i in vocab and nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ') else w2v.wv.get_vector(i)  for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([np.array(vocab[i])*mul_factor if (i in vocab.keys() and nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ') else np.array(vocab[i]) for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.30, random_state=42)
result_table = []

In [14]:
epochs = 20

In [20]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")


Epoch 1/20
49/49 [==============================] - 50s 1s/step - loss: 1.3778 - accuracy: 0.3133
Epoch 2/20
49/49 [==============================] - 50s 1s/step - loss: 1.3471 - accuracy: 0.3333
Epoch 3/20
49/49 [==============================] - 50s 1s/step - loss: 1.3077 - accuracy: 0.3637
Epoch 4/20
49/49 [==============================] - 50s 1s/step - loss: 1.2792 - accuracy: 0.3737
Epoch 5/20
49/49 [==============================] - 50s 1s/step - loss: 1.2708 - accuracy: 0.3700
Epoch 6/20
49/49 [==============================] - 49s 1s/step - loss: 1.2599 - accuracy: 0.3843
Epoch 7/20
49/49 [==============================] - 49s 1s/step - loss: 1.2547 - accuracy: 0.3880
Epoch 8/20
49/49 [==============================] - 49s 999ms/step - loss: 1.2510 - accuracy: 0.3892
Epoch 9/20
49/49 [==============================] - 49s 1s/step - loss: 1.2572 - accuracy: 0.3608
Epoch 10/20
49/49 [==============================] - 49s 1s/step - loss: 1.2567 - accuracy: 0.3641
Epoch 11/20
49/4

In [21]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_BI")

Epoch 1/20
49/49 [==============================] - 50s 1s/step - loss: 1.3629 - accuracy: 0.3314
Epoch 2/20
49/49 [==============================] - 50s 1s/step - loss: 1.2940 - accuracy: 0.3976
Epoch 3/20
49/49 [==============================] - 50s 1s/step - loss: 1.2325 - accuracy: 0.4355
Epoch 4/20
49/49 [==============================] - 50s 1s/step - loss: 1.1915 - accuracy: 0.4506
Epoch 5/20
49/49 [==============================] - 50s 1s/step - loss: 1.1611 - accuracy: 0.4698
Epoch 6/20
49/49 [==============================] - 50s 1s/step - loss: 1.1378 - accuracy: 0.4782
Epoch 7/20
49/49 [==============================] - 50s 1s/step - loss: 1.1199 - accuracy: 0.4929
Epoch 8/20
49/49 [==============================] - 50s 1s/step - loss: 1.1051 - accuracy: 0.4967
Epoch 9/20
49/49 [==============================] - 50s 1s/step - loss: 1.0945 - accuracy: 0.5006
Epoch 10/20
49/49 [==============================] - 50s 1s/step - loss: 1.1117 - accuracy: 0.4937
Epoch 11/20
49/49 [

In [26]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_sg_rnn = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history_sg_rnn)

result_table+=[history_sg_rnn[1]]
print("SG_RNN")

Epoch 1/20
49/49 [==============================] - 53s 1s/step - loss: 1.3778 - accuracy: 0.3076
Epoch 2/20
49/49 [==============================] - 53s 1s/step - loss: 1.3684 - accuracy: 0.3284
Epoch 3/20
49/49 [==============================] - 54s 1s/step - loss: 1.3590 - accuracy: 0.3349
Epoch 4/20
49/49 [==============================] - 54s 1s/step - loss: 1.3480 - accuracy: 0.3445
Epoch 5/20
49/49 [==============================] - 55s 1s/step - loss: 1.3471 - accuracy: 0.3478
Epoch 6/20
49/49 [==============================] - 54s 1s/step - loss: 1.3395 - accuracy: 0.3555
Epoch 7/20
49/49 [==============================] - 53s 1s/step - loss: 1.3350 - accuracy: 0.3606
Epoch 8/20
49/49 [==============================] - 54s 1s/step - loss: 1.3227 - accuracy: 0.3696
Epoch 9/20
49/49 [==============================] - 55s 1s/step - loss: 1.3263 - accuracy: 0.3610
Epoch 10/20
49/49 [==============================] - 54s 1s/step - loss: 1.3406 - accuracy: 0.3561
Epoch 11/20
49/49 [

In [27]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_LSTM")

Epoch 1/20
49/49 [==============================] - 50s 1s/step - loss: 1.3779 - accuracy: 0.3176
Epoch 2/20
49/49 [==============================] - 50s 1s/step - loss: 1.3498 - accuracy: 0.3245
Epoch 3/20
49/49 [==============================] - 50s 1s/step - loss: 1.3286 - accuracy: 0.3627
Epoch 4/20
49/49 [==============================] - 50s 1s/step - loss: 1.3014 - accuracy: 0.3729
Epoch 5/20
49/49 [==============================] - 50s 1s/step - loss: 1.2821 - accuracy: 0.3773
Epoch 6/20
49/49 [==============================] - 50s 1s/step - loss: 1.2761 - accuracy: 0.3814
Epoch 7/20
49/49 [==============================] - 50s 1s/step - loss: 1.2723 - accuracy: 0.3765
Epoch 8/20
49/49 [==============================] - 50s 1s/step - loss: 1.2683 - accuracy: 0.3792
Epoch 9/20
49/49 [==============================] - 50s 1s/step - loss: 1.2624 - accuracy: 0.3786
Epoch 10/20
49/49 [==============================] - 50s 1s/step - loss: 1.2574 - accuracy: 0.3845
Epoch 11/20
49/49 [

In [28]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_BI")

Epoch 1/20
49/49 [==============================] - 50s 1s/step - loss: 1.3627 - accuracy: 0.3329
Epoch 2/20
49/49 [==============================] - 50s 1s/step - loss: 1.2874 - accuracy: 0.3969
Epoch 3/20
49/49 [==============================] - 50s 1s/step - loss: 1.2398 - accuracy: 0.4278
Epoch 4/20
49/49 [==============================] - 50s 1s/step - loss: 1.1996 - accuracy: 0.4443
Epoch 5/20
49/49 [==============================] - 50s 1s/step - loss: 1.1723 - accuracy: 0.4582
Epoch 6/20
49/49 [==============================] - 51s 1s/step - loss: 1.1483 - accuracy: 0.4755
Epoch 7/20
49/49 [==============================] - 51s 1s/step - loss: 1.1293 - accuracy: 0.4794
Epoch 8/20
49/49 [==============================] - 51s 1s/step - loss: 1.1058 - accuracy: 0.4951
Epoch 9/20
49/49 [==============================] - 51s 1s/step - loss: 1.0910 - accuracy: 0.5080
Epoch 10/20
49/49 [==============================] - 52s 1s/step - loss: 1.0798 - accuracy: 0.5053
Epoch 11/20
49/49 [

In [15]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_RNN")

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
49/49 [==============================] - 60s 1s/step - loss: 1.3720 - accuracy: 0.3151
Epoch 2/20
49/49 [==============================] - 60s 1s/step - loss: 1.3516 - accuracy: 0.3529
Epoch 3/20
49/49 [==============================] - 60s 1s/step - loss: 1.3539 - accuracy: 0.3465
Epoch 4/20
49/49 [==============================] - 60s 1s/step - loss: 1.3645 - accuracy: 0.3412
Epoch 5/20
49/49 [==============================] - 60s 1s/step - loss: 1.3689 - accuracy: 0.3392
Epoch 6/20
49/49 [==============================] - 60s 1s/step - loss: 1.3850 - accuracy: 0.2863
Epoch 7/20
49/49 [==============================] - 60s 1s/step - loss: 1.3768 - accuracy: 0.3027
Epoch 8/20
49/49 [==============================] - 60s 1s/step - loss: 1.3753 - accuracy: 0.3143
Epoch 9/20
49/49 [==============================] - 60s 1s/step - loss: 1.3734 - accuracy: 0.3182
Epoch 10/20
49/49 [=======================

In [26]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_LSTM")


Epoch 1/20
49/49 [==============================] - 18s 365ms/step - loss: 1.3759 - accuracy: 0.3206
Epoch 2/20
49/49 [==============================] - 18s 368ms/step - loss: 1.3198 - accuracy: 0.3620
Epoch 3/20
49/49 [==============================] - 18s 370ms/step - loss: 1.2347 - accuracy: 0.4276
Epoch 4/20
49/49 [==============================] - 18s 368ms/step - loss: 1.1893 - accuracy: 0.4427
Epoch 5/20
49/49 [==============================] - 18s 368ms/step - loss: 1.1607 - accuracy: 0.4535
Epoch 6/20
49/49 [==============================] - 18s 367ms/step - loss: 1.1346 - accuracy: 0.4661
Epoch 7/20
49/49 [==============================] - 18s 367ms/step - loss: 1.1169 - accuracy: 0.4778
Epoch 8/20
49/49 [==============================] - 18s 369ms/step - loss: 1.0964 - accuracy: 0.4890
Epoch 9/20
49/49 [==============================] - 18s 376ms/step - loss: 1.0739 - accuracy: 0.5020
Epoch 10/20
49/49 [==============================] - 18s 373ms/step - loss: 1.0530 - accura

In [17]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_BI")

Epoch 1/20
49/49 [==============================] - 20s 417ms/step - loss: 1.3638 - accuracy: 0.3241
Epoch 2/20
49/49 [==============================] - 21s 419ms/step - loss: 1.2828 - accuracy: 0.4131
Epoch 3/20
49/49 [==============================] - 20s 418ms/step - loss: 1.1704 - accuracy: 0.4922
Epoch 4/20
49/49 [==============================] - 20s 414ms/step - loss: 1.0832 - accuracy: 0.5512
Epoch 5/20
49/49 [==============================] - 20s 413ms/step - loss: 1.0115 - accuracy: 0.5890
Epoch 6/20
49/49 [==============================] - 20s 408ms/step - loss: 0.9492 - accuracy: 0.6286
Epoch 7/20
49/49 [==============================] - 20s 416ms/step - loss: 0.8880 - accuracy: 0.6549
Epoch 8/20
49/49 [==============================] - 20s 412ms/step - loss: 0.8390 - accuracy: 0.6759
Epoch 9/20
49/49 [==============================] - 20s 410ms/step - loss: 0.8060 - accuracy: 0.6873
Epoch 10/20
49/49 [==============================] - 20s 410ms/step - loss: 0.7646 - accura

In [23]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_RNN")

Epoch 1/20
49/49 [==============================] - 17s 342ms/step - loss: 1.3825 - accuracy: 0.3004
Epoch 2/20
49/49 [==============================] - 17s 344ms/step - loss: 1.3648 - accuracy: 0.3276
Epoch 3/20
49/49 [==============================] - 17s 341ms/step - loss: 1.3332 - accuracy: 0.3645
Epoch 4/20
49/49 [==============================] - 17s 340ms/step - loss: 1.3011 - accuracy: 0.3880
Epoch 5/20
49/49 [==============================] - 17s 342ms/step - loss: 1.2700 - accuracy: 0.4076
Epoch 6/20
49/49 [==============================] - 17s 340ms/step - loss: 1.2559 - accuracy: 0.4145
Epoch 7/20
49/49 [==============================] - 17s 340ms/step - loss: 1.2420 - accuracy: 0.4218
Epoch 8/20
49/49 [==============================] - 17s 345ms/step - loss: 1.2397 - accuracy: 0.4292
Epoch 9/20
49/49 [==============================] - 17s 342ms/step - loss: 1.2334 - accuracy: 0.4337
Epoch 10/20
49/49 [==============================] - 17s 341ms/step - loss: 1.2205 - accura

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg.add(tf.keras.layers.LSTM(64))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))

In [25]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg.add(tf.keras.layers.LSTM(64))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/20
49/49 [==============================] - 50s 1s/step - loss: 1.3788 - accuracy: 0.3184
Epoch 2/20
49/49 [==============================] - 50s 1s/step - loss: 1.3747 - accuracy: 0.3206
Epoch 3/20
49/49 [==============================] - 50s 1s/step - loss: 1.3737 - accuracy: 0.3206
Epoch 4/20
49/49 [==============================] - 50s 1s/step - loss: 1.3515 - accuracy: 0.3355
Epoch 5/20
49/49 [==============================] - 50s 1s/step - loss: 1.3083 - accuracy: 0.3706
Epoch 6/20
49/49 [==============================] - 50s 1s/step - loss: 1.3064 - accuracy: 0.3682
Epoch 7/20
49/49 [==============================] - 51s 1s/step - loss: 1.3049 - accuracy: 0.3665
Epoch 8/20
49/49 [==============================] - 52s 1s/step - loss: 1.3207 - accuracy: 0.3588
Epoch 9/20
49/49 [==============================] - 51s 1s/step - loss: 1.3516 - accuracy: 0.3314
Epoch 10/20
49/49 [==============================] - 51s 1s/step - loss: 1.3125 - accuracy: 0.3645
Epoch 11/20
49/49 [